# Instructions
The following code was designed in order to load in and score motion/freezing data from mpg/wmv video files. It allows for cropping of the video frame to reduce influence of miniscope/optogenetic cables.  After initally loading in the video, the user is able to crop the video frame by adjusting a single variable.  Motion is then detected by capturing the number of pixels whose frame by frame grayscale change value exceeds a threshold (determined using FreezeAnalysis_Calibration.ipynb).  Freezing is then assessed by calculating when motion drops below a user-defined threshold.  The user is able to visualize raw video, animal motion, and whether an animal is deemed to be freezing in order optimize parameters.  Final output is a csv which provides frame by frame motion and freezing across the session.  With the exception of setting the file path and small number of parameters (ie frames per second in video, freezing threshold), user can run through code.  Once parameters are found that work well, batch processing of multiple videos can be performed using FreezeAnalysis_BatchProcess.ipynb.

### Package Requirements
The following will need to be installed in your Conda environment:

- python (3.6.5)

- jupyter

- imread

- mahotas(1.4.4)

- numpy(1.14.3)

- pandas(0.23.0)

- matplotlib(2.2.2)

- opencv(3.4.3)

The following commands can be executed in your terminal to create an environment with these packages: 

- conda config --add channels conda-forge

- conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 opencv=3.4.3 jupyter imread


# 1. Load Necessary Packages

In [ ]:
import pylab 
import os
import cv2
import numpy as np
import mahotas as mh 
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# 2. User Sets Directory and File Information

In [ ]:
dpath = "/Users/ZP/Videos # directory containing file
file = "Test_m6.mpg" #filename.  will take mpg and wmv files but maybe more.  only mpg1 has been extensively tested.
fps = 30 #frames per second

# 3. Load Video Information.  Display First Frame

In [ ]:
#Upoad file
fpath = dpath + "/" + file
print('file: '+ fpath)
cap = cv2.VideoCapture(fpath)

#Get maxiumum frame of file. Note that this is updated later if fewer frames detected
cap_max = int(cap.get(7)) #7 is index of total frames
print('total frames: ' + str(cap_max))

#Set first frame to be displayed
cap.set(1,0) #first index references frame property, second specifies next frame to grab

#Initialize ycrop in the event that it is not subsequently set to something other than 0
ycrop = 0

#Load first frame
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Present frame
plt.gray()
plt.figure(figsize=(8,8))
plt.title('first frame: ' + str(0))
plt.imshow(gray)

# 4. User Selects Cropping Parameters
### Here, upper portion of video to be cropped can be set.  Use y-axis value from previous cell. Be sure to set ycrop=0 if no cropping desired


In [ ]:
#set ycrop
ycrop = 140

#Displays result
plt.gray()
plt.figure(figsize=(8,8))
plt.title('Cropped Frame. ycrop=' + str(ycrop))
plt.imshow(gray[ycrop:,:])

# Analyze Motion Across Session

## 5a. User Sets Parameters for Motion Analysis

In [ ]:
mt_cutoff = 10.3 #grayscale difference value required for pixel to be counted as changing
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 

## 5b. Detect Motion
###### Here, code loops through all frames and detects number of pixels whose grayscale change exceeds cutoff per frame.  Motion vector is populated with these values.

In [ ]:
Display_Vid = False #set to True if you want to view motion detection video.  Otherwise set to False.  While helpful to validate code is working optimally, slows things down.  Smaller segment of video can be run after freezing is measured.

#Initialize vector to store motion values in
Motion = np.zeros(cap_max)

#Re-initialize video
cap.set(1,0) #first index references frame property, second specifies next frame to grab

#Initialize first frame
ret, frame = cap.read()
frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
frame_new = frame_new[ycrop:,:]
frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA)

#Loop through frames to detect frame by frame differences
for x in range (1,cap_max):
    
    #Reset old frame
    frame_old = frame_new
    
    #Attempt to load next frame
    ret, frame = cap.read()
    if ret == True:
        
        #Reset new frame and process
        frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_new = frame_new[ycrop:,:]
        frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA) # used to reduce influence of jitter from one frame to the next
    
        #Calculate difference between frames
        frame_dif = np.absolute(frame_new - frame_old)
        frame_cut = frame_dif > mt_cutoff
        frame_cut = frame_cut.astype('uint8')
        
        #Display pixels detected as exceeeding threshold
        if Display_Vid:
            cv2.imshow("preview", frame_cut*255)
            cv2.waitKey(1)
 
        #Assign difference to array
        Motion[x]=np.sum(frame_cut)
        
    else: 
        #if no frame is detected
        cap_max = (x-1) #Reset max frame to last frame detected
        Motion = Motion[:cap_max] #Amend length of motion vector
        print('Maximum frames less than expected. Adjusting to: ' + str(cap_max))
        break

#Close video window if open        
if Display_Vid:
    cv2.destroyAllWindows()
    _=cv2.waitKey(1)   
    

## 5c. Plot Motion Across Session

In [ ]:
plt.figure(figsize=(15,4))
plt.ylim(0,2000)
plt.title('Motion Across Session')
plt.xlabel('Frame')
plt.ylabel('Pixel Change')
plt.plot(Motion)

# Analyze Session Freezing

## 6a. User Selects Freezing Parameters

In [ ]:
FreezeThresh = 200 #Upper bound for freezing, in frame-by-frame pixels changed
MinDuration = 1 #Number of seconds motion must be below threshold to begin accruing freezing
MinDuration = MinDuration * fps

## 6b. Measure Freezing

In [ ]:
#Find frames below thresh
BelowThresh = (Motion<FreezeThresh).astype(int)

#Perform local cumulative thresh detection
#For each consecutive frame motion is below threshold count is increased by 1 until motion goes above thresh, at which point coint is set back to 0
CumThresh = np.zeros(cap_max)
for x in range (1,cap_max):
    if (BelowThresh[x]==1):
        CumThresh[x] = CumThresh[x-1] + BelowThresh[x]
        
#Measure Freezing
Freezing = (CumThresh>=MinDuration).astype(int) #whenever motion has dropped below the thresh for at least MinDuration call this freezing

#the following code makes it so that the initial 30 before CumThresh meets MinDuration are still counted as freezing
#Debating whether the code renders better results
for x in range( len(Freezing) - 2, -1, -1) : 
    if Freezing[x] == 0 and Freezing[x+1]>0 and Freezing[x+1]<MinDuration:
        Freezing[x] = Freezing[x+1] + 1
Freezing = (Freezing>0).astype(int)

## 6c. Plot Freezing

In [ ]:
print('Average Freezing: '+ str(np.average(Freezing)*100) + ' %')

#Plot Freezing
plt.figure(figsize=(15,4))
plt.ylim(0,1.1)
plt.title('Freezing Across Session')
plt.xlabel('Frame')
plt.ylabel('Freezing')
plt.plot(Freezing)

# Display section of video and display when freezing is occuring
##### After analyzing freezing a section of the video can be replayed and the animal's state - "Active/Freezing" - will be displayed.  

## 7a. User Selects Portion of Video to be Played

In [ ]:
start = 10 #start point of video segment in seconds.  0 if beginning of video.
end = 20 #end point of video segment in seconds.  this is NOT the duration of the segment

#Video parameters
img_scale = 2.5 # #Scale image by this number.5 will reduce original video size 1/2, 2 will double it, etc.
save_video=False #Option to save video if desired.  Currently will be saved at 20 fps even if video is something else

## 7b. Play video.  Press 'q' to quite video

In [ ]:
#redfine start/end in frames
fstart = start*fps
fend = end*fps

#set play speed and define first frame
rate = int(1000/fps) #duration each frame is present for, in milliseconds
cap.set(1,fstart) #set reference position of first frame to 0

#set text parameters
textfont = cv2.FONT_HERSHEY_SIMPLEX
textposition = (10,30)
textfontscale = 1
textlinetype = 2

#Initialize first frame
ret, frame = cap.read()
frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
frame_new = frame_new[ycrop:,:]
frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA)

#Initialize video storage if desired
if save_video:
    width = int(frame.shape[1]*img_scale)
    height = int((frame.shape[0]+frame.shape[0]-ycrop)*img_scale)
    fourcc = cv2.VideoWriter_fourcc(*'jpeg') #only writes up to 20 fps, though video read can be 30.
    #fourcc = cv2.VideoWriter_fourcc(*'MJPG') #only writes up to 20 fps, though video read can be 30.
    writer = cv2.VideoWriter('out.avi', fourcc, 20.0, (width, height),isColor=False)

#Loop through frames to detect frame by frame differences
for x in range (fstart+1,fend):
    
    # press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    #Attempt to load next frame
    ret, frame = cap.read()
    if ret == True:
        
        #Convert to gray scale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #set old frame to new frame
        frame_old = frame_new
          
        #Reset new frame and process
        frame_new = frame
        frame_new = frame_new[ycrop:,:]
        frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA) # used to reduce influence of jitter from one frame to the next
        
        #Calculate difference between frames
        frame_dif = np.absolute(frame_new - frame_old)
        frame_cut = frame_dif > mt_cutoff
        frame_cut = frame_cut.astype('uint8')*255
        
        #Add text to videos
        if Freezing[x]==1:
            texttext = 'FREEZING'
            textfontcolor = 255
        else:
            texttext = 'ACTIVE'
            textfontcolor = 0
        cv2.putText(frame,texttext,textposition,textfont,textfontscale,textfontcolor,textlinetype)

        #Display video
        frame = cv2.resize(frame, (0,0), fx=img_scale, fy=img_scale) 
        frame_cut = cv2.resize(frame_cut, (0,0), fx=img_scale, fy=img_scale)
        preview = np.concatenate((frame,frame_cut))
        cv2.imshow("preview",preview)
        cv2.waitKey(rate)
        
        #Save video (if desired). 
        if save_video:
            writer.write(preview) 

    else: 
        print('No frame detected at frame : ' + str(x) + '.Stopping video play')
        break
    
#Close video window and video writer if open        
cv2.destroyAllWindows()
_=cv2.waitKey(1) 
if save_video:
    writer.release()


# 8. Create Dataframe and Save

In [ ]:
#Set output name
fpath_out = fpath[:-4] + '.FreezingOutput.csv'

#Create Dataframe
DataFrame = pd.DataFrame(
    {'File': [file]*cap_max,
     'FPS': np.ones(cap_max)*fps,
     'MotionCutoff':np.ones(cap_max)*mt_cutoff,
     'FreezeThresh':np.ones(cap_max)*FreezeThresh,
     'Frame': np.arange(cap_max),
     'Motion': Motion,
     'Freezing': Freezing
    })   

DataFrame.to_csv(fpath_out)